In [18]:
import numpy as np


In [38]:
# network size N
N = 5

# number of edges
M = N*3

## initialize time
T = 0
## choose Tmax
Tmax = 5

# network size N
N = 5

# number of edges
M = N*3

Tmax = 5


In [47]:
# network size N
N = 5

# number of edges
M = N*3

Tmax = 5

class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

## inputs are the initial condition and the edge matrix
def find_tranformation_matrix(b, E):

    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    X = np.identity(N, dtype=bool)
    
    vf = np.vectorize(lambda x: 2**x)
    ints = np.array(list(range(N)))
    I = vf(ints)

    counts = Counter()
    for element in I:
        counts.increment(element, 1)
    # number of non-zero rows
    z = np.ones(N, dtype=bool)

    ## initialize the number of distinct rows to N (start with Identity matrix)
    distinct_rows = N

    # run simulation
    for t in range(n):
        
        ## recovery event occurs (turns diagonal entry to 0 and premultiplies A) == row i of A is set to False 
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            print(f"recovery event at iteration {t} of node {i}")
        
            row_i_not_distinct = I[i] in counts and counts[I[i]] > 1

            ## if the row is already all zeros, nothing needs to be done (no change in distinct rows)
            if z[i] == True:

                X[i, :] = False

                z[i] == False

                I[i] = 0

                counts.increment(I[i], -1)
                counts.increment(0, 1)

                if 0 in counts and counts[0] >= 1:
                    ## if there is at least one non zero row, and the row being zeroed was distinct, then the number of distinct rows decreases by 1
                    if row_i_not_distinct == False:

                        distinct_rows -= 1
                    ## if there is at least one non zero row and the row being zeroed was not distinct, then the number of distinct rows remains the same
                     
                else:
                    ## if there are no nonzero rows, and the row being zeroed was not distinct, then the number of distinct rows increases by 1
                    if row_i_not_distinct == True:
                            
                        distinct_rows = min(distinct_rows + 1, N)
                    ## if there are no nonzero rows, and the row being zeroed was distinct, then the number of distinct rows remains the same

        else:
        ## spreading event (turns symmetric entries to 1 and premultiplies A) == row i and j of A is set to the OR of row i and j
        ## check if the two rows being ORed are all zero rows, if yes, nothing to be done; else, OR the rows and set corresponding entries in Z to True

            i,j = E[np.random.randint(M)]

            print(f"spreading event at iteration {t} of nodes {i} and {j}")

            if I[i] != 0 or I[j] != 0:

                ## nothing happens if two rows being ORed are two zero rows

                if I[i] != I[j]:
                    
                    ## no change in distinct rows if the two rows being ORed are the same
                    # have confirmed that row A and row B are not the same and at least one of them is nonzero
                    
                    new = int(''.join('1' if x else '0' for x in np.logical_or(X[i], X[j]))[::-1], 2)
                    new_not_distinct = new in counts and counts[new] >= 1
                    print(new_not_distinct)
                    row_i_not_distinct = I[i] in counts and counts[I[i]] > 1
                    print(row_i_not_distinct)
                    row_j_not_distinct = I[j] in counts and counts[I[j]] > 1
                    print(row_j_not_distinct)

                    ## if the row being created is distinct, but the rows being collapsed were also distinct, then the number of distinct rows is one less
                    # if the row being created is distinct, but ONE of the rows being collapsed was distinct, then the number of distinct rows is the same
                    ## if the row being created is distinct, but NEITHER of the rows collapsed were distinct, then the number of distinct rows is increasing by 1
                    if new_not_distinct == False:
                        if row_i_not_distinct == False and row_j_not_distinct == False:
                            distinct_rows -= 1
                        if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                            distinct_rows = distinct_rows
                        if row_i_not_distinct == True and row_j_not_distinct == True:
                            distinct_rows = min(distinct_rows + 1, N)
                    ## if the row being created is not distinct, but BOTH the rows being collapsed were, then the number of distinct rows decreases by 2
                    ## if the row being created is not distinct, but ONE of the rows being collapsed was, then the number of distinct rows decreases by 1
                    ## if the row being created is not distinct, but NEITHER of the rows being collapsed were, then the number of distinct rows remains the same
                    else:
                        if row_i_not_distinct == False and row_j_not_distinct == False:
                            distinct_rows -= 2
                        if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                            distinct_rows -= 1
                    
                ## progres the simulation normally
                X[i] = X[j] = np.logical_or(X[i], X[j])
                z[i] = z[j] = True
                counts.increment(I[i], -1)
                counts.increment(I[j], -1)
                counts.increment(new, 1)
                I[i] = I[j] = new
                          
        print(distinct_rows)    
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2)
        if distinct_rows == 2:
            
            break

    return X, I





In [48]:
b = 0.5

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

x0 = np.ones(N, dtype=bool)
matrix_A = find_tranformation_matrix(b, E)
print(matrix_A)


recovery event at iteration 0 of node 3
4
spreading event at iteration 1 of nodes 3 and 3
4
spreading event at iteration 2 of nodes 2 and 3
True
False
False
2
(array([[ True, False, False, False, False],
       [False,  True, False, False, False],
       [False, False,  True, False, False],
       [False, False,  True, False, False],
       [False, False, False, False,  True]]), array([ 1,  2,  4,  4, 16]))


In [21]:
# Your decimal number
decimal_number = 5

# The length of the binary string
length = 8

# Convert the decimal number to a binary string of a specific length
binary_string = format(decimal_number, '0' + str(length) + 'b')
binary_array = np.array([int(bit) for bit in binary_string])
print(binary_array)



[0 0 0 0 0 1 0 1]


In [26]:
vf = np.vectorize(lambda x: 2**x)
X = np.array(list(range(N)))
I = vf(X)
print(I)


[ 1  2  4  8 16]


In [37]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

counts = Counter()
for element in I:
    counts.increment(element, 1)

counts.increment(1, 1)
print(len(counts))
print(counts)
print(counts[1])


I[1]
row_i_not_distinct = I[1] in counts and counts[I[1]] > 1
print(row_i_not_distinct)

counts[0] >= 1


5
{2: 1, 4: 1, 8: 1, 16: 1, 1: 2}
2
False


KeyError: 0